In this repository, we test how good the fixed point caluclation is and is the dynamical system remains stable for this arbitrary recurrent weight matrix.

In [1]:
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
import time
import math
from perturbed_organics.spectrum_general import matrix_solution
from perturbed_organics.spectrum_general import sim_solution
import perturbed_organics.model.ORGaNICs_models as organics
import perturbed_organics.utils as utils
import numpy as np
from joblib import Parallel, delayed, parallel_backend
import argparse
import os
from matplotlib import colors
from scipy.optimize import fsolve, curve_fit
from scipy import integrate
import warnings
import shutil
import json

torch.set_default_dtype(torch.float64)
parser = argparse.ArgumentParser(description="Sparse Matrix Stability Scan")

In [2]:
# Set default parameters (replacing argparse)
task_id = 0
num_tasks = 1
print(f"Task ID: {task_id}")

# Model parameters
model_name = "delocalized"
matrix_type = "goe_symmetric"
input_scale = "log-scale"
data_save_loc = ""         # Default value (adjust as needed)
extra_file_name = ""       # Default value (adjust as needed)
initial_type = "norm"
N = 100
s = 100
mu = 0.0

# Simulation parameters
num_trials = 10
num_delta = 1
num_input = 10
min_delta = 0.05
max_delta = 5.0
min_input = 0.01
max_input = 5.0

# Define the scan parameters
delta_range = np.linspace(min_delta, max_delta, num_delta)
if input_scale == "log-scale":
    input_range = np.logspace(np.log10(min_input), np.log10(max_input), num_input)
else:
    input_range = np.linspace(min_input, max_input, num_input)

# %%
device = torch.device("cpu")

# Define the parameters of the ORGaNICs
params = {"N_y": N, "N_a": N, "eta": 0.02, "noise_type": "additive"}
b0 = 1.0 * torch.ones(N)
b1 = 1.0 * torch.ones(N)
sigma = torch.tensor([0.1])
tauA = 0.002 + 0 * torch.abs(torch.randn(N) * 0.001)
tauY = 0.002 + 0 * torch.abs(torch.randn(N) * 0.001)
Way = torch.ones(N, N)

# Create list of parameter combinations
param_combinations = [(i, j) for i in range(num_delta) for j in range(num_input)]

# Split parameter combinations among tasks
param_chunks = np.array_split(param_combinations, num_tasks)
my_params = param_chunks[task_id]

Task ID: 0


In [3]:
def run_trial(i, j, k, delta, input):
    # Initialize variables for the trial

    z = utils.make_input_drive(N=N, input_type=model_name, input_norm=input)

    Wyy = torch.eye(N) + utils.generate_matrix(
        N=N, matrix_type=matrix_type, s=s, mu=mu, delta=delta
    )

    # find the spectral radius of Wyy
    eigvals = torch.linalg.eigvals(Wyy)
    spectral_radius = torch.max(torch.abs(eigvals))

    # Instantiate the model
    model = organics.ORGaNICs2Dgeneral(
        params=params,
        b0=b0,
        b1=b1,
        sigma=sigma,
        tauA=tauA,
        tauY=tauY,
        Wyy=Wyy,
        Way=Way,
        z=z,
        initial_type=initial_type,
        run_jacobian=False,
    )

    y_s0, a_s0 = model.analytical_ss()
    y_s1, a_s1 = model.first_order_correction()

    #### run adaptive simulation scheme and find the dynamics

    # define the time vector
    tau_min = min(torch.min(tauA), torch.min(tauY))
    tau_max= max(torch.max(tauA), torch.max(tauY))
    chunk_time = 100 * tau_max  # Simulate in chunks of 100 * tau_max
    dt = 0.05 * tau_min # defines the timestep of simulation
    points = int(chunk_time / dt)  # Number of points per chunk
    t_chunk = torch.linspace(0, chunk_time, points)

    max_loops = 20

    condition = 0 # 0: not met, 1: diverging, 2: fixed point, 3: periodic

    # define the model for simulation
    y0 = model.inital_conditions(initial_type=initial_type)
    sim_obj = sim_solution(model)

    for loop in range(max_loops):
        traj_segment = sim_obj.simulate(t_chunk, y0=y0)
        y0 = traj_segment[-1, :]

        if utils.is_diverging(traj_segment):
            condition = 1
            break
        elif utils.is_fixed_point(traj_segment):
            condition = 2
            break
        elif utils.is_periodic(traj_segment[:, 0].detach().numpy()):
            condition = 3
            break

    if condition == 2:
        # find the jacobian and its eigenvalue
        ss = traj_segment[-1, :]
        J, _ = model.jacobian_autograd(ss=ss)
        eigvals_J = torch.linalg.eigvals(J)
        return (
            condition,
            spectral_radius.to(torch.float16),
            y_s0.to(torch.float16),
            a_s0.to(torch.float16),
            ss[0:N].to(torch.float16),
            ss[N : 2 * N].to(torch.float16),
            y_s1.to(torch.float16),
            a_s1.to(torch.float16),
            eigvals_J,
        )
    else:
        return (
            condition,
            spectral_radius.to(torch.float16),
            y_s0.to(torch.float16),
            a_s0.to(torch.float16),
            None,
            None,
            y_s1.to(torch.float16),
            a_s1.to(torch.float16),
            None,
        )
        

def run_trial_and_collect(i, j, k):
    delta = delta_range[i]
    input = input_range[j]
    condition, spec_radius, y_s0, a_s0, y_s_actual, a_s_actual, y_s1, a_s1, eigvals_J = (
        run_trial(i, j, k, delta, input)
    )
    return (
        i,
        j,
        k,
        condition,
        spec_radius,
        y_s0,
        a_s0,
        y_s_actual,
        a_s_actual,
        y_s1,
        a_s1,
        eigvals_J,
    )


In [4]:


results = Parallel(n_jobs=-1, verbose=2)(
    delayed(run_trial_and_collect)(i, j, k)
    for i, j in my_params
    for k in range(num_trials)
)

# Collect and save partial results
condition_task = torch.full(
    (num_delta, num_input, num_trials), fill_value=-1, dtype=torch.int8
)
spectral_radius_task = torch.full(
    (num_delta, num_input, num_trials), fill_value=float("nan"), dtype=torch.float16
)
norm_fixed_point_y_task = torch.full(
    (num_delta, num_input, num_trials, N), fill_value=float("nan"), dtype=torch.float16
)
norm_fixed_point_a_task = torch.full(
    (num_delta, num_input, num_trials, N), fill_value=float("nan"), dtype=torch.float16
)
actual_fixed_point_y_task = torch.full(
    (num_delta, num_input, num_trials, N), fill_value=float("nan"), dtype=torch.float16
)
actual_fixed_point_a_task = torch.full(
    (num_delta, num_input, num_trials, N), fill_value=float("nan"), dtype=torch.float16
)
first_order_perturb_y_task = torch.full(
    (num_delta, num_input, num_trials, N), fill_value=float("nan"), dtype=torch.float16
)
first_order_perturb_a_task = torch.full(
    (num_delta, num_input, num_trials, N), fill_value=float("nan"), dtype=torch.float16
)
eigvals_J_task = torch.full(
    (num_delta, num_input, num_trials, 2 * N),
    fill_value=float("nan"),
    dtype=torch.complex64,
)

# Update the results arrays
for res in results:
    (
        i,
        j,
        k,
        condition,
        spec_radius,
        y_s0,
        a_s0,
        y_s_actual,
        a_s_actual,
        y_s1,
        a_s1,
        eigvals_J,
    ) = res
    condition_task[i, j, k] = condition
    spectral_radius_task[i, j, k] = spec_radius
    norm_fixed_point_y_task[i, j, k] = y_s0
    norm_fixed_point_a_task[i, j, k] = a_s0
    first_order_perturb_y_task[i, j, k] = y_s1
    first_order_perturb_a_task[i, j, k] = a_s1

    if y_s_actual is not None:
        actual_fixed_point_y_task[i, j, k] = y_s_actual
        actual_fixed_point_a_task[i, j, k] = a_s_actual
        eigvals_J_task[i, j, k] = eigvals_J


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   14.8s finished
